# IMPORT LIBRARIES

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install underthesea
!pip install transformers
!pip install vncorenlp
!pip install emoji

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.9/20.9 MB 34.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 993.5/993.5 kB 23.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 657.8/657.8 kB 26.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 42.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 24.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 72.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 64.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 24.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 19.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for vncorenlp: filename=vncorenlp-1.0.3-py3-none-any.whl size=2645932 sha256=9cf055ad61eec963114356ffcb6ea33e551195e5938f080f818022f6ff210d26
  Stored in directory: /root/.cache/pip/wheels/5d/d9/b3/41f6c6b1ab758561

In [ ]:
!mkdir -p vncorenlp/models/wordsegmenter
!wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/VnCoreNLP-1.2.jar
!wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/models/wordsegmenter/vi-vocab
!wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/models/wordsegmenter/wordsegmenter.rdr
!mv VnCoreNLP-1.2.jar vncorenlp/
!mv vi-vocab vncorenlp/models/wordsegmenter/
!mv wordsegmenter.rdr vncorenlp/models/wordsegmenter/


--2023-11-06 03:59:25--  https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/VnCoreNLP-1.2.jar
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 27412703 (26M) [application/octet-stream]
Saving to: ‘VnCoreNLP-1.2.jar’

VnCoreNLP-1.2.jar   100%[===================>]  26.14M   142MB/s    in 0.2s    

2023-11-06 03:59:26 (142 MB/s) - ‘VnCoreNLP-1.2.jar’ saved [27412703/27412703]

--2023-11-06 03:59:26--  https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/models/wordsegmenter/vi-vocab
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 

In [1]:
from vncorenlp import VnCoreNLP


In [2]:
import json
import numpy as np
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer,AutoModel
import re
import emoji
import torch

In [4]:
class DataSource():
    def __init__(self,file_path):
        self.file_path = file_path
        file_status = open(file_path,'r',encoding='utf-8-sig')
        self.read_file = file_status.readlines()

        target = "data/Hotel_target.txt"

        self.read_target =  open(target,'r',encoding='utf-8-sig').readlines()
        self.targets = np.array(list(map(lambda x:x.strip(),self.read_target)))

        self.texts = []
        self.labels = []

        for i in range(1,len(self.read_file),4):
            self.texts.append(self.read_file[i].replace('\n',''))            
            self.labels.append(self.read_file[i+1].replace('\n',''))
        file_status.close()
        
    def getDataFrame(self):
        dummy = [[0 for j in range(len(self.targets)+1)] for i in range(len(self.texts))]

        def TextConverted(text):
                if text == 'neutral':
                    return 1
                elif text == 'negative':
                    return 2
                elif text == 'positive':
                    return 3
                else:
                    return 0

        for idx in range(len(self.texts)):
            r = re.findall("#",self.labels[idx])
            temp = self.labels[idx]
            start = 0
            end = len(self.labels[idx])
   
            for i in range(len(r)):
                temp = self.labels[idx][start:end]
                s = temp[re.search("{.*?}",temp).start()+1:re.search("{.*?}",temp).end()-1].split(',')
                aspect = s[0]
                sentiment = s[1][1:]
                arg_aspect = np.argwhere(aspect == self.targets)
                dummy[idx][arg_aspect[0][0]+1] = int(TextConverted(sentiment))
                start += re.search("{.*?}",temp).end()

        data = pd.DataFrame(dummy,columns=np.append("Sentences",self.targets))
        data['Sentences'] = self.texts

        rs = {}
        rs['comment'] = data['Sentences'].values
        rs['label'] = data.iloc[:,1:].values
        return rs

In [5]:
lbb = ['FACILITIES#CLEANLINESS', 'FACILITIES#COMFORT',
       'FACILITIES#DESIGN&FEATURES', 'FACILITIES#GENERAL',
       'FACILITIES#MISCELLANEOUS', 'FACILITIES#PRICES', 'FACILITIES#QUALITY',
       'FOOD&DRINKS#MISCELLANEOUS', 'FOOD&DRINKS#PRICES',
       'FOOD&DRINKS#QUALITY', 'FOOD&DRINKS#STYLE&OPTIONS', 'HOTEL#CLEANLINESS',
       'HOTEL#COMFORT', 'HOTEL#DESIGN&FEATURES', 'HOTEL#GENERAL',
       'HOTEL#MISCELLANEOUS', 'HOTEL#PRICES', 'HOTEL#QUALITY',
       'LOCATION#GENERAL', 'ROOMS#CLEANLINESS', 'ROOMS#COMFORT',
       'ROOMS#DESIGN&FEATURES', 'ROOMS#GENERAL', 'ROOMS#MISCELLANEOUS',
       'ROOMS#PRICES', 'ROOMS#QUALITY', 'ROOM_AMENITIES#CLEANLINESS',
       'ROOM_AMENITIES#COMFORT', 'ROOM_AMENITIES#DESIGN&FEATURES',
       'ROOM_AMENITIES#GENERAL', 'ROOM_AMENITIES#MISCELLANEOUS',
       'ROOM_AMENITIES#PRICES', 'ROOM_AMENITIES#QUALITY', 'SERVICE#GENERAL']

In [6]:
data = DataSource("data/Hotel_dev.txt")
df = data.getDataFrame()
df

{'comment': array(['Chưa có thang máy. Chưa chấp nhận thanh toán bằng thẻ. Địa điểm dễ tìm, bày trí bằng tre nứa rất mát mẻ, bạn lễ tân nhiệt tình, niềm nở, thân thiện, tốt bụng cực kỳ. Tôi đặt phòng 1 giường đôi nhưng biết tôi đi cùng 2 con nhỏ nên ks đã chủ động chuẩn bị thêm 1 chiếc giường tầng cho 2 bé. Phòng rộng rãi, bày trí đẹp, giường êm, nhà vệ sinh cực sạch sẽ. Vị trí thuận lợi, dễ di chuyển. Bạn lễ tân đã giới thiệu tôi các điểm ăn ngon trên đường Lê Thị Riêng, Nguyễn Trãi gần đó. Tóm lại, Nguyễn Shack xứng đáng để tôi giới thiệu bạn bè.',
        'Chúng tôi không cảm thấy thoải mái vì ở chỉ 1 ngày mà cúp điện 3,4 lần . Thang máy thì quá nóng và quá chậm chạp. Bữa sáng tuyệt. Phòng ngủ tiện nghi đẹp. Nhân viên phục phụ rất tốt.',
        'Nhân viên rất nhiệt tình thân thiện, luôn giúp đỡ mỗi khi tôi cần. Giường ngủ êm, thoải mái, không gian phòng rộng rãi. Rất thích đồ ăn của Nhà hàng Vỹ Dạ.',
        ..., 'Phục vụ chu đáo, chỗ nghỉ thoải lái, phòng sạch đẹp',
        'Nhan 

# TEXT PREPROCESSING

In [7]:
class TextNormalize:
    def __init__(self):
        self.vowels_to_ids = {}
        self.vowels_table = [
            ['a', 'à', 'á', 'ả', 'ã', 'ạ', 'a' ],
            ['ă', 'ằ', 'ắ', 'ẳ', 'ẵ', 'ặ', 'aw'],
            ['â', 'ầ', 'ấ', 'ẩ', 'ẫ', 'ậ', 'aa'],
            ['e', 'è', 'é', 'ẻ', 'ẽ', 'ẹ', 'e' ],
            ['ê', 'ề', 'ế', 'ể', 'ễ', 'ệ', 'ee'],
            ['i', 'ì', 'í', 'ỉ', 'ĩ', 'ị', 'i' ],
            ['o', 'ò', 'ó', 'ỏ', 'õ', 'ọ', 'o' ],
            ['ô', 'ồ', 'ố', 'ổ', 'ỗ', 'ộ', 'o'],
            ['ơ', 'ờ', 'ớ', 'ở', 'ỡ', 'ợ', 'ow'],
            ['u', 'ù', 'ú', 'ủ', 'ũ', 'ụ', 'u' ],
            ['ư', 'ừ', 'ứ', 'ử', 'ữ', 'ự', 'uw'],
            ['y', 'ỳ', 'ý', 'ỷ', 'ỹ', 'ỵ', 'y' ]
        ]
        pass

    def createVowelsTable(self):
        """Create Vowels Table"""
        for i in range(len(self.vowels_table)):
            for j in range(len(self.vowels_table[i]) - 1):
                self.vowels_to_ids[self.vowels_table[i][j]] = (i, j)

    def IsValidVietnameseWord(self,word):
        """Nguyên âm chỉ có thể đứng chung với nguyên âm. Một từ không thể có 2 nguyên âm cách nhau bởi 1 phụ âm"""
        chars = list(word)
        #nguyen am
        vowel_index = -1
        for i in range(len(chars)):
            idx_vowel_table = self.vowels_to_ids.get(chars[i],(-1,-1))[0]
            if idx_vowel_table != -1:
                if vowel_index == -1:
                    vowel_index = i
                else:
                    if i - vowel_index != 1:
                        return False
                    vowel_index = i
        return True

    def WordStandardized(self,word):
        """Standardize Word"""
        if not self.IsValidVietnameseWord(word):
            return word

        chars = list(word)
        vowel_indexes = []

        # tìm vị trí nguyên âm
        qu_or_gi = False
        thanh_dieu = 0
        for i in range(len(chars)):
            vowel_table_row, vowel_table_col = self.vowels_to_ids.get(chars[i],(-1,-1))
            if vowel_table_row == -1 :
                continue
            # qu
            if vowel_table_row == 9:
                if i != 0 and chars[i-1] == 'q':
                    chars[i] = 'u'
                    qu_or_gi = True
            # gi
            elif vowel_table_row == 5:
                if i != 0 and chars[i-1] == 'g':
                    chars[i] = 'i'
                    qu_or_gi = True

            # có chứa thanh điệu
            if vowel_table_col != 0:
                thanh_dieu = vowel_table_col
                chars[i] = self.vowels_table[vowel_table_row][0]

            vowel_indexes.append(i)
        # 1 nguyên âm
        if len(vowel_indexes) == 1:
            c = chars[vowel_indexes[0]]
            chars[vowel_indexes[0]] = self.vowels_table[self.vowels_to_ids[c][0]][thanh_dieu]
            return ''.join(chars)

        for idx_vowel in vowel_indexes:
            vowel_table_row, vowel_table_col = self.vowels_to_ids.get(chars[idx_vowel],(-1,-1))
            #ê, ơ, ô
            if vowel_table_row == 4 or vowel_table_row == 7 or vowel_table_row == 8:
                c = chars[idx_vowel]
                chars[idx_vowel] = self.vowels_table[self.vowels_to_ids[c][0]][thanh_dieu]
                return ''.join(chars)

            # kiểm tra qu và gi, 2-3 nguyên âm thì nguyên âm thứ 2 chứa dấu
            if qu_or_gi:
                if len(vowel_indexes) == 2 or len(vowel_indexes) == 3:
                    c = chars[vowel_indexes[1]]
                    chars[vowel_indexes[1]] = self.vowels_table[self.vowels_to_ids[c][0]][thanh_dieu]
                return ''.join(chars)

            # 2 nguyên âm
            if len(vowel_indexes) == 2:
                # âm cuối là nguyên âm
                if vowel_indexes[-1] == len(chars) - 1:
                    c = chars[vowel_indexes[0]]
                    chars[vowel_indexes[0]] = self.vowels_table[self.vowels_to_ids[c][0]][thanh_dieu]
                else:
                    c = chars[vowel_indexes[-1]]
                    chars[vowel_indexes[-1]] = self.vowels_table[self.vowels_to_ids[c][0]][thanh_dieu]
                return ''.join(chars)

            elif len(vowel_indexes) == 3:
                # âm cuối là nguyên âm
                if vowel_indexes[-1] == len(chars) - 1:
                    c = chars[vowel_indexes[1]]
                    chars[vowel_indexes[1]] = self.vowels_table[self.vowels_to_ids[c][0]][thanh_dieu]
                else:
                    c = chars[vowel_indexes[-1]]
                    chars[vowel_indexes[-1]] = self.vowels_table[self.vowels_to_ids[c][0]][thanh_dieu]
                return ''.join(chars)

        return ''.join(chars)

    def normalize(self,text):

        #Chuyen sang viet thuong
        text = text.lower()

        # Rút gọn từ kéo dài
        text = re.sub(r'(\w)\1+',r'\1',text)

        # xóa các emoji dư thừa
        text = emoji.replace_emoji(text, replace='')  # no emoji

        text = text.split()
        # chuẩn hóa thanh điệu
        for i in range(len(text)):
            text[i] = self.WordStandardized(text[i])

        text = ' '.join(text)

        # xóa space d
        text = re.sub(r"( )\1+",r'\1',text)
        text = re.sub(r"[:)^@!`~%;?(\+\-\'\"]+",r'',text)

        # remove hastag
        text = re.sub("(@[A-Za-z0-9]+)|(#[0-9A-Za-z]+)"," ", text)
        return text

In [8]:
def convert_unicode(text):
  char1252 = 'à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ'
  charutf8 = 'à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ'
  char1252 = char1252.split('|')
  charutf8 = charutf8.split('|')

  dic = {}
  for i in range(len(char1252)): dic[char1252[i]] = charutf8[i]
  return re.sub(
      r'à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ',
      lambda x: dic[x.group()], text
)


In [9]:
class TextGen():
    def __init__(self, json_path, tokenizer_path, use_segmenter = False):
        self.json_path = json_path
        with open(self.json_path,'r',encoding = 'utf-8') as file:
            self.json_object = json.load(file)

        self.use_segmenter = use_segmenter
        self.tokenizer = AutoTokenizer.from_pretrained(tokenizer_path)
        self.normalize = TextNormalize().normalize

        self.ASPECT = ["Facilities", "Service", "Public_area", "Location", "Food", "Room"]
        self.POLARITY = ['None','Negative', "Neutral", "Positive"]

        # self.ASPECT_POLARITY = []
        # for aspect in self.ASPECT:
        #     for polarity in self.POLARITY:
        #         self.ASPECT_POLARITY.append(f"{aspect}#{polarity}")

    def convert_to_onehot_label(self,labels):
        onehot_labels = []

        for label in labels:
            init_labels = [0]*len(self.ASPECT)
            if len(label) == 0:
                onehot_labels.append(init_labels)
            else:
                for l in label:
                    asp, pol = l.split("#")
                    asp_idx = self.ASPECT.index(asp)
                    pol_idx = self.POLARITY.index(pol)
                    init_labels[asp_idx] = pol_idx
                onehot_labels.append(init_labels)
        return onehot_labels

    def get_comment_and_label(self):
        """
            INPUT: json_path
            OUTPUT: comment, label
        """

        comment = []
        label = []

        for obj in self.json_object:
            # have label
            cmt = convert_unicode(obj['comment'])
            if cmt == "":
              continue
            if "type" in obj:
                obj_label = [obj['type']]
                if "choices" in obj_label[0]:
                    obj_label = obj_label[0]['choices']
                label.append(obj_label)
            else:
                label.append([])
            comment.append(cmt)


        return comment, label

    def get_data_dict(self):

        data_dict = {}

        comments, labels = self.get_comment_and_label()

        data_dict['raw_comment'] = comments
        data_dict['raw_label'] = labels

        onehot_label = self.convert_to_onehot_label(labels)
        data_dict['label'] = onehot_label

        cmt_tokenize = []
        normalize_comments = list(map(lambda x: self.normalize(x),comments))
        if self.use_segmenter:
          rdrsegmenter = VnCoreNLP("vncorenlp/VnCoreNLP-1.2.jar", annotators="wseg", max_heap_size='-Xmx500m')
          normalize_comments = list(map(lambda x: " ".join([" ".join(sent) for sent in rdrsegmenter.tokenize(x)]), normalize_comments))

        data_dict['comment'] = normalize_comments
        data_dict['label'] = onehot_label

        return data_dict

In [11]:
PRETRAINED_PATH = "vinai/phobert-base"

In [12]:
from vncorenlp import VnCoreNLP
rdrsegmenter = VnCoreNLP("vncorenlp/VnCoreNLP-1.2.jar", annotators="wseg", max_heap_size='-Xmx500m')


FileNotFoundError: File "vncorenlp/VnCoreNLP-1.2.jar" was not found, please check again.

In [10]:
text_dataset = TextGen("/content/drive/MyDrive/tma/label_data.json",PRETRAINED_PATH,True)
train_dict = text_dataset.get_data_dict()

NameError: name 'PRETRAINED_PATH' is not defined

In [ ]:
ASPECT = text_dataset.ASPECT
ASPECT

['Facilities', 'Service', 'Public_area', 'Location', 'Food', 'Room']

In [54]:
comments, labels = train_dict['comment'], train_dict['label']

In [55]:
for comment, label in zip(comments,labels):
    print(comment)
    print(label)
    print("=============================")

Chưa có thang máy. Chưa chấp nhận thanh toán bằng thẻ. Địa điểm dễ tìm, bày trí bằng tre nứa rất mát mẻ, bạn lễ tân nhiệt tình, niềm nở, thân thiện, tốt bụng cực kỳ. Tôi đặt phòng 1 giường đôi nhưng biết tôi đi cùng 2 con nhỏ nên ks đã chủ động chuẩn bị thêm 1 chiếc giường tầng cho 2 bé. Phòng rộng rãi, bày trí đẹp, giường êm, nhà vệ sinh cực sạch sẽ. Vị trí thuận lợi, dễ di chuyển. Bạn lễ tân đã giới thiệu tôi các điểm ăn ngon trên đường Lê Thị Riêng, Nguyễn Trãi gần đó. Tóm lại, Nguyễn Shack xứng đáng để tôi giới thiệu bạn bè.
[0 0 2 0 0 0 0 0 0 0 0 0 3 3 3 0 0 0 3 0 0 3 0 0 0 0 3 0 0 0 0 0 3 3]
Chúng tôi không cảm thấy thoải mái vì ở chỉ 1 ngày mà cúp điện 3,4 lần . Thang máy thì quá nóng và quá chậm chạp. Bữa sáng tuyệt. Phòng ngủ tiện nghi đẹp. Nhân viên phục phụ rất tốt.
[0 2 0 0 0 0 2 0 0 3 0 0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 3 0 0 0 0 3]
Nhân viên rất nhiệt tình thân thiện, luôn giúp đỡ mỗi khi tôi cần. Giường ngủ êm, thoải mái, không gian phòng rộng rãi. Rất thích đồ ăn của Nh

In [ ]:
class MyDataset(Dataset):
  def __init__(self, data_dict,tokenizer,max_len):
    self.data_dict = data_dict
    self.tokenizer = tokenizer
    self.max_len = max_len

  def __len__(self):
    return len(self.data_dict['comment'])

  def __getitem__(self,idx):
    comment = self.data_dict['comment'][idx]
    label = self.data_dict['label'][idx]

    tokenize = self.tokenizer(comment,padding='max_length', max_length = 150,truncation=True,return_tensors='pt')
    # rs = []
    for key in self.tokenizer.model_input_names:
      tokenize[key] = tokenize[key][0]

    # rs.append(label)
    tokenize['label'] = torch.tensor(label).view(-1,1)

    # for k in tokenize.keys():
    #   tokenize[k] = torch.LongTensor(tokenize[k])
    return tokenize


In [ ]:
tokenizer = AutoTokenizer.from_pretrained(PRETRAINED_PATH)

In [ ]:
dataset = MyDataset(train_dict, tokenizer, 150)

In [ ]:
# def my_collate_fn(data):
#   label = []
#   temp = {}
#   for key in data[0].keys():
#     if key != 'label':
#       temp[key] = []

#   for i in range(len(data)):
#     for key in data[i].keys():
#       if key != "label":
#         temp[key].append(data[i][key])
#       else:
#         label.append(data[i]['label'])

#   # for key in temp.keys():
#   #   temp[key] = torch.LongTensor(temp[key])
#   return temp, label

In [ ]:
train_loader = DataLoader(dataset, batch_size = 16, shuffle = False, )

In [ ]:
# for data in train_loader:
#   temp = data.copy()
#   label = temp['label']
#   del temp['label']
#   print(temp)
#   print(label)
#   break

# MODEL

In [ ]:
class FeatureExtractor(torch.nn.Module):
  def __init__(self, pretrained_path):
    super(FeatureExtractor,self).__init__()
    self.cell = AutoModel.from_pretrained(pretrained_path)

  def forward(self, input):
    seq_out, pooled_out = self.cell(**input)[:2]

    return pooled_out

In [ ]:
class MyModel(torch.nn.Module):
  def __init__(self,pretrained_path,num_classes,num_polarity):
    super(MyModel, self).__init__()
    self.feature_extractor = FeatureExtractor(pretrained_path)
    self.linear_each_class = torch.nn.ModuleList([torch.nn.Linear(768, num_polarity) for i in range(num_classes)])

  def forward(self, input):
    cls_token = self.feature_extractor(input) # (batch_size,1)

    batch_size = cls_token.size()[0]

    out = []
    for i in range(len(self.linear_each_class)):
      out.append(self.linear_each_class[i](cls_token))

    out = torch.cat(out, dim = 1).view(batch_size, len(ASPECT),4)

    return out

In [ ]:
#device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
model = MyModel("vinai/phobert-base",len(ASPECT),4)
model = model.to(device)

In [ ]:
NUM_EPOCHS = 20

In [ ]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(),lr = 2e-5)

In [ ]:
def load_model(path):
    check_point = torch.load(path,map_location=torch.device('cpu'))
    return check_point

In [ ]:
def save_model(path, model, epoch):
    torch.save({
        "epoch": epoch,
        "model_state_dict": model.state_dict(),
    },path)

In [ ]:
from tqdm import tqdm

def train_one_epoch(model, optimizer, criterion, dataloader, epoch):
  epoch_loss = 0
  n_iter = len(dataloader)

  with tqdm(train_loader, unit="batch") as tepoch:
    for data in tepoch:
      tepoch.set_description(f"Epoch {epoch}")

      input = data.copy()
      label = input['label']
      del input['label']

      input = input.to(device)
      label = label.to(device)
      logits = model(input)

      loss = 0
      for asp_idx in range(len(ASPECT)):
        l = criterion(logits[:,asp_idx,:],label[:,asp_idx,:].squeeze(-1))
        loss += l

      epoch_loss += loss.item()
      loss.backward()
      torch.nn.utils.clip_grad_norm_(model.parameters(),1.0)
      optimizer.step()
      optimizer.zero_grad()

      tepoch.set_postfix(loss=loss.item())

    epoch_loss /= n_iter
  print(f"At EPOCH {epoch}, loss = {epoch_loss}")
  print("======================================")

  return epoch_loss

In [ ]:
from tqdm import tqdm

train_losses = []
for epoch in range(NUM_EPOCHS):
  train_loss = train_one_epoch(model, optimizer, criterion, train_loader, epoch)
  train_losses.append(train_loss)

Epoch 0: 100%|██████████| 7/7 [00:02<00:00,  2.54batch/s, loss=6.64]


At EPOCH 0, loss = 7.563623428344727


Epoch 1: 100%|██████████| 7/7 [00:02<00:00,  2.62batch/s, loss=5.28]


At EPOCH 1, loss = 6.115422316959926


Epoch 2: 100%|██████████| 7/7 [00:02<00:00,  2.61batch/s, loss=4.48]


...

At EPOCH 17, loss = 1.636538531099047


Epoch 18: 100%|██████████| 7/7 [00:02<00:00,  2.58batch/s, loss=0.864]


At EPOCH 18, loss = 1.5253929155213493


Epoch 19: 100%|██████████| 7/7 [00:02<00:00,  2.55batch/s, loss=0.799]

At EPOCH 19, loss = 1.4401413117136275


In [ ]:
save_model("/content/drive/MyDrive/tma/text_model.pth",model,epoch)

# INFERENCE

In [ ]:
def load_model(path):
    check_point = torch.load(path,map_location=torch.device('cpu'))
    return check_point

In [ ]:
PRETRAINED_PATH = "vinai/phobert-base"
rdrsegmenter = VnCoreNLP("vncorenlp/VnCoreNLP-1.2.jar", annotators="wseg", max_heap_size='-Xmx500m')
tnormalize = TextNormalize()
tokenizer = AutoTokenizer.from_pretrained(PRETRAINED_PATH)

In [ ]:
ASPECT = ["Facilities", "Service", "Public_area", "Location", "Food", "Room"]
POLARITY = ['None','Negative', "Neutral", "Positive"]


In [ ]:
device = "cuda" if torch.cuda.is_available() else 'cpu'
checkpoint = load_model("/content/drive/MyDrive/tma/text_model.pth")


In [ ]:
model = MyModel(PRETRAINED_PATH,len(ASPECT),4)
model.load_state_dict(checkpoint['model_state_dict'])
model = model.to(device)

In [ ]:
comment = "Lần đầu trải nghiệm Terracotta, và sẽ còn tiếp tục quay lại cho những lần tới, khuôn viên và không gian rất phù hợp cho gia đình có trẻ em và người già. Các bạn nhân viên từ lễ tân đến dịch vụ đặt chỗ và các bạn hỗ trợ dọn dẹp tại phòng ở trong thời gian lưu trú rất là dễ thương và thân thiện. Chúc Terracotta ngày càng phát triển mạnh hơn nhé!"
comment = convert_unicode(comment)
comment = tnormalize.normalize(comment)
comment = " ".join([ " ".join(x) for x in rdrsegmenter.tokenize(comment)])
comment

'lần đầu trải_nghiệm teracota , và sẽ còn tiếp_tục quay lại cho những lần tới , khuôn_viên và không_gian rất phù_hợp cho gia_đình có trẻ_em và người già . các bạn nhân_viên từ lễ_tân đến dịch_vụ đặt chỗ và các bạn hỗ_trợ dọn_dẹp tại phòng ở trong thời_gian lưu_trú rất là dễ_thương và thân_thiện . chúc teracota ngày_càng phát_triển mạnh hơn nhé'

In [ ]:
comment_tokenize = tokenizer(comment,padding='max_length', max_length = 150,truncation=True,return_tensors='pt')
comment_tokenize

{'input_ids': tensor([[    0,   101,   127,  1756,  4055, 19990,   237,     4,     6,    38,
            50,   194,   588,    44,    13,    21,   101,    75,     4,  5256,
             6,  1152,    59,   549,    13,   161,    10,   861,     6,    18,
          1687,     5,     9,    88,   650,    39, 17112,    30,   399,   320,
           683,     6,     9,    88,   291,  5801,    35,   292,    25,    12,
           130,  6567,    59,     8,  4924,     6,  3188,     5,  3788,  4055,
         19990,   237,   734,   134,   384,    48,  2083,     2,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,  

In [ ]:
model_output = model(comment_tokenize.to(device))

In [ ]:
_, y_pred = torch.max(torch.nn.functional.softmax(model_output,dim=-1),dim=-1)
y_pred = y_pred[0]
y_pred

tensor([0, 3, 3, 0, 0, 0])

In [ ]:
text_y_pred = []

for i in range(len(y_pred)):
  if y_pred[i] != 0:
    asp = ASPECT[i]
    pol = POLARITY[y_pred[i]]
    text_y_pred.append(f"{asp}#{pol}")

text_y_pred

['Service#Positive', 'Public_area#Positive']